In [1]:
import os
import torch
import random
import datetime
import pandas as pd
import numpy as np

from torch.utils.data import Dataset
from src.datasets import RL4RS, ContentWise, DummyData, OpenCDP
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems, get_svd_encoder
from src.embeddings import RecsysEmbedding

experiment_name = 'InSlateAttention'
device = 'cuda:2'
seed = 123
pkl_path = '../pkl/'

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)


KeyboardInterrupt



In [ ]:
torch.__version__

# Модель

In [ ]:
import torch.nn.functional as F
torch.autograd.set_detect_anomaly(True)

class AttentionResponseModel(torch.nn.Module):
    """
    No recurrent dependency, just slate-wise attention.
    """
    def __init__(self, embedding, nheads=2, output_dim=1):
        super().__init__()
        self.embedding_dim = embedding.embedding_dim
        self.embedding = embedding
        self.attention= torch.nn.MultiheadAttention(
            2 * embedding.embedding_dim,
            num_heads=nheads,
            batch_first=True
        )
        
        self.out_layer = torch.nn.Linear(2 * embedding.embedding_dim, output_dim)

    def forward(self, batch):
        item_embs, user_embs = self.embedding(batch)
        shp = item_embs.shape
        
        # let model attent to first padd token if slate is empty to avoid NaN gradients
        # (anyway they does not contrinute into metrics computation)
        key_padding_mask = batch['slates_mask'].clone()
        key_padding_mask[:,:, 0] = True 
        
        # repeating user embedding for each item embeding
        features = torch.cat(
            [
                item_embs,
                user_embs[:, :, None, :].repeat(1, 1, shp[-2], 1).reshape(shp)
            ],
            dim = -1
        )
        
        # all recomendations in session are scored independently
        features = features.flatten(0,1)
        
        features, attn_map = self.attention(
            features, features, features,
            key_padding_mask=~key_padding_mask.flatten(0, 1)
        )
        
        # transforming back to sequence shape
        shp = list(shp)
        shp[-1] *= 2
        return self.out_layer(features.reshape(shp)).squeeze(-1)

# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [ ]:
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

model = AttentionResponseModel(
    RecsysEmbedding(d.n_items, dummy_matrix, embeddings='neural').to('cpu'),
    output_dim=1
).to('cpu')

train(
    model, 
    dummy_loader, dummy_loader, dummy_loader,
    device=device, lr=1e-3, num_epochs=5000, dummy=True,
    silent=True,
)


# ContentWise

In [ ]:
content_wise_results = []
dataset = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=150)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

In [ ]:
for embeddings in ['svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    
    model = AttentionResponseModel(
        RecsysEmbedding(train_num_items, train_user_item_matrix, embeddings=embeddings),
        output_dim=1
    ).to(device)

    _, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)

In [ ]:
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items

# RL4RS

In [ ]:
rl4rs_results = []
dataset = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=350)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

In [ ]:
for embeddings in ['neural','explicit', 'svd',  ]:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

    model = AttentionResponseModel(
        RecsysEmbedding(
            train_num_items, 
            train_user_item_matrix, 
            embeddings=embeddings,
            embedding_dim=40
        ),
        output_dim=1
    ).to(device)

    best_model, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
        silent=True
    )
    
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)

In [ ]:
pd.DataFrame(rl4rs_results).to_csv(f'results/rl4rs_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items

# OpenCDP

In [ ]:
for group in ['cosmetics', 'multi']:
    for filename in os.listdir(pkl_path):    
        result = []
        if not filename.startswith(group):
            continue
        print(f"\n == {filename} ==")
        dataset = OpenCDP.load(os.path.join(pkl_path, filename))
        (
            train_loader, 
            val_loader,
            test_loader, 
            train_user_item_matrix, 
            train_num_items
        ) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=800)
    
        print(f"{len(dataset)} data points among {len(train_loader)} batches")
        for embeddings in ['neural', 'svd']:
            print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

            model = AttentionResponseModel(
                RecsysEmbedding(train_num_items, train_user_item_matrix, embeddings=embeddings),
                output_dim=1
            ).to(device)

            best_model, metrics = train(
                model, 
                train_loader, val_loader, test_loader, 
                device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
                silent=True
            )
            
            print(metrics)
            metrics['embeddings'] = embeddings
            result.append(metrics)
        pd.DataFrame(result).to_csv(f'results/{filename}_{experiment_name}.csv')
        del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items